In [1]:
import numpy as np
import pandas as pd

import sqlite3
import warnings
warnings.filterwarnings('ignore')
database = "database.sqlite"

In [2]:
conn = sqlite3.connect(database)
tables = pd.read_sql("""
select * from sqlite_master
where type='table';
""", conn)
display(tables)

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,4,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,Player_Attributes,Player_Attributes,11,"CREATE TABLE ""Player_Attributes"" (\n\t`id`\tIN..."
2,table,Player,Player,14,CREATE TABLE `Player` (\n\t`id`\tINTEGER PRIMA...
3,table,Match,Match,18,CREATE TABLE `Match` (\n\t`id`\tINTEGER PRIMAR...
4,table,League,League,24,CREATE TABLE `League` (\n\t`id`\tINTEGER PRIMA...
5,table,Country,Country,26,CREATE TABLE `Country` (\n\t`id`\tINTEGER PRIM...
6,table,Team,Team,29,"CREATE TABLE ""Team"" (\n\t`id`\tINTEGER PRIMARY..."
7,table,Team_Attributes,Team_Attributes,2,CREATE TABLE `Team_Attributes` (\n\t`id`\tINTE...
8,table,netflix_data,netflix_data,305672,"CREATE TABLE ""netflix_data"" (\n""index"" INTEGER..."
9,table,netflix_content,netflix_content,307031,"CREATE TABLE ""netflix_content"" (\n""index"" INTE..."


1. `SELECT` statement: Retrieve the names of all countries from the "Country" table.

In [ ]:
qry1 = pd.read_sql("""
SELECT name 
FROM Country;
""", conn)

display(qry1)

2. Retrieve the names of all leagues from the "League" table for the country with the name 'Spain'.

In [54]:
qry2 = pd.read_sql("""
select name
from league
where country_id = (
select id
from country
where name = 'Spain'
)
""", conn)

display(qry2)

,name
0,Spain LIGA BBVA


3. Retrieve the match details (match_api_id, home_team_goal, away_team_goal) along with the names of the home team and away team for the matches played in the '2015/2016' season

In [9]:
qry3 = pd.read_sql("""
select M.match_api_id, M.home_team_goal, M.away_team_goal, Th.team_long_name, Ta.team_long_name
from Match M
left join Team Th on M.home_team_api_id = Th.team_api_id
left join Team Ta on M.away_team_api_id = Ta.team_api_id
where season = '2015/2016'
order by M.home_team_goal ;
""", conn)

display(qry3)

,match_api_id,home_team_goal,away_team_goal,team_long_name
0,1979907,0,4,RSC Anderlecht
1,1979911,0,1,KRC Genk
2,1979914,0,4,Royal Excel Mouscron
3,1979917,0,2,Waasland-Beveren
4,1979927,0,1,Sporting Charleroi
...,...,...,...,...
3321,2216671,7,0,Motherwell
3322,2030431,7,1,RC Celta de Vigo
3323,1992185,7,0,Lugano
3324,1990939,8,1,Hamilton Academical FC


4. `GROUP BY` statement: Retrieve the total number of goals scored by each team in the "Match" table, grouped by the country and league they belong to.

In [87]:
qry4 = pd.read_sql("""
select T.team_long_name as team_name, sum(M.home_team_goal + M.away_team_goal) as total_goals, L.name as league_name, C.name as cntry_name
from Team T
join Match M
on T.team_api_id = M.home_team_api_id
join Country C
on C.id = M.country_id
join League L
on L.id = M.league_id
group by C.name, L.name, T.team_long_name;

""", conn)

display(qry4)

,team_name,total_goals,league_name,cntry_name
0,Beerschot AC,210,Belgium Jupiler League,Belgium
1,Club Brugge KV,330,Belgium Jupiler League,Belgium
2,FCV Dender EH,44,Belgium Jupiler League,Belgium
3,KAA Gent,326,Belgium Jupiler League,Belgium
4,KAS Eupen,37,Belgium Jupiler League,Belgium
...,...,...,...,...
291,FC Zürich,413,Switzerland Super League,Switzerland
292,Grasshopper Club Zürich,396,Switzerland Super League,Switzerland
293,Lugano,56,Switzerland Super League,Switzerland
294,Neuchâtel Xamax,207,Switzerland Super League,Switzerland


5. `HAVING` statement: Retrieve the average number of goals scored per match for teams that have played at least 10 matches in the "Match" table.

In [5]:
qry2 = pd.read_sql("""
select avg(M.home_team_goal + M.away_team_goal) as avg_goals, T.team_long_name
from Match M
join Team T
on M.home_team_api_id = T.team_api_id
group by T.team_long_name
having count(match_api_id) >= 10;
""", conn)

display(qry2)



,avg_goals,team_long_name
0,2.735294,1. FC Kaiserslautern
1,2.764706,1. FC Köln
2,2.776471,1. FC Nürnberg
3,2.848739,1. FSV Mainz 05
4,2.789474,AC Ajaccio
...,...,...
291,2.894737,Xerez Club Deportivo
292,2.388889,Zagłębie Lubin
293,2.766667,Zawisza Bydgoszcz
294,2.539474,Évian Thonon Gaillard FC


6. `ORDER BY` statement: Retrieve the names of players and their heights from the "Player" table, sorted in descending order of height.

In [ ]:
qry6 = pd.read_sql("""
select player_name, height 
from player order by height desc;
""", conn)

display(qry6)

7. `LIMIT` statement: Retrieve the top 10 teams
    with the highest number of goals scored in a
    match from the "Team" table.

In [98]:
qry7 = pd.read_sql("""
select T.team_long_name, sum(M.home_team_goal + M.away_team_goal) as total_goalss
from Team T
left join Match M
on T.id = M.id
group by T.team_long_name
order by total_goalss desc
limit 10;
""", conn)
display(qry7)

,team_long_name,total_goalss
0,Pescara,7
1,Hamburger SV,7
2,Évian Thonon Gaillard FC,6
3,Valenciennes FC,6
4,SV Werder Bremen,6
5,Middlesbrough,6
6,Le Havre AC,6
7,Girondins de Bordeaux,6
8,FC Ingolstadt 04,6
9,Burnley,6


8. `DISTINCT` keyword: Retrieve the unique seasons from the "Match" table.

In [4]:
qry102 = pd.read_sql("""
select distinct(season)
from Match
""", conn)
display(qry102)

,season
0,2008/2009
1,2009/2010
2,2010/2011
3,2011/2012
4,2012/2013
5,2013/2014
6,2014/2015
7,2015/2016


9. `NULL` value detection:  Retrieve the names
    of players from the "Player" table whose
    height is not recorded (NULL)

In [100]:
qry9 = pd.read_sql("""
select player_name
from Player
where height IS NULL
""", conn)
display(qry9)

,player_name


10. Subquery: Retrieve the names of players from the "Player" table who have a higher height than the overall average height of all players

In [103]:
qry10 = pd.read_sql("""
select player_name, height
from Player
where height > (
select avg(height)
from Player)
order by height;
""", conn)

display(qry10)

,player_name,height
0,Aaron Appindangoye,182.88
1,Aaron Galindo,182.88
2,Aaron Hughes,182.88
3,Aaron Hunt,182.88
4,Aaron Mokoena,182.88
...,...,...
5865,Pietro Marino,203.20
5866,Stefan Maierhofer,203.20
5867,Vanja Milinkovic-Savic,203.20
5868,Zeljko Kalac,203.20


11. `BETWEEN` operator: Retrieve the matches from the "Match" table where the number of goals scored by the home team is between 3 and 5 (inclusive)

In [ ]:
qry11 = pd.read_sql("""
select M.home_team_goal as goals, T.team_long_name
from match M
left join team T
on M.id = T.id
where M.home_team_goal between 3 and 5;
""", conn)
display(qry11)

12. `LIKE` operator: Retrieve the names of teams from the "Team" table whose long name starts with 'FC'

In [ ]:
qry102 = pd.read_sql("""
select team_long_name
from Team
where team_long_name like 'FC%'
;
""", conn)
display(qry102)

13. `COUNT()` function: Retrieve the number of matches played in each league from the "Match" table

In [111]:
qry13 = pd.read_sql("""
select count(*) as no_of_match, L.name
from Match M
right join League L
on M.league_id = L.id
group by L.name
;
""", conn)

display(qry13)

,no_of_match,name
0,1728,Belgium Jupiler League
1,3040,England Premier League
2,3040,France Ligue 1
3,2448,Germany 1. Bundesliga
4,3017,Italy Serie A
5,2448,Netherlands Eredivisie
6,1920,Poland Ekstraklasa
7,2052,Portugal Liga ZON Sagres
8,1824,Scotland Premier League
9,3040,Spain LIGA BBVA


14. `MAX()` function: Retrieve the player name and the highest height from the "Player" table

In [119]:
qry14 = pd.read_sql("""
select max(height) as highest_hight, player_name
from Player
;
""", conn)
display(qry14)

,highest_hight,player_name
0,208.28,Kristof van Hout


15. `MIN()` function:  Retrieve the player name and the lowest weight from the "Player" table

In [121]:
qry15 = pd.read_sql("""
select min(weight) as lowest_weight, player_name
from Player
;
""", conn)
display(qry15)

,lowest_weight,player_name
0,117,Juan Quero


16. `SUM()` function: Retrieve the total number of goals scored by each team in the "Match" table

In [133]:
qry16 = pd.read_sql("""
select sum(M.home_team_goal + M.away_team_goal) as total_goals, T.team_long_name
from Match M
join Team T
on M.id = T.id
group by T.team_long_name
;
""", conn)
display(qry16)

,total_goals,team_long_name
0,2,1. FC Kaiserslautern
1,4,1. FC Köln
2,4,1. FC Nürnberg
3,5,1. FSV Mainz 05
4,4,AC Ajaccio
...,...,...
150,3,West Bromwich Albion
151,3,West Ham United
152,0,Wigan Athletic
153,3,Wolverhampton Wanderers


17. `AVG()` function: Retrieve the average weight of players in the "Player" table

In [135]:
qry14 = pd.read_sql("""
select avg(weight) as avg_weight
from Player
;
""", conn)
display(qry14)

,avg_weight
0,168.380289


18. `IN` operator: Retrieve the names of teams 
    from the "Team" table that have played matches
    in either '2012/2013' or '2013/2014' seasons

In [17]:
qry15 = pd.read_sql("""
select distinct(T.team_long_name), M.season
from team T
join match M
on T.team_api_id = M.home_team_api_id
where M.season IN ('2012/2013' , '2013/2014');
""", conn)
qry15

,team_long_name,season
0,KV Kortrijk,2012/2013
1,Beerschot AC,2012/2013
2,RAEC Mons,2012/2013
3,Standard de Liège,2012/2013
4,KV Mechelen,2012/2013
...,...,...
355,Grasshopper Club Zürich,2013/2014
356,FC Aarau,2013/2014
357,FC Thun,2013/2014
358,FC Sion,2013/2014


19. `JOIN` with multiple tables: Retrieve all the matches played

In [11]:
qry19 = pd.read_sql("""
select m.*, l.name AS league_name, T.team_long_name
from match M
join league l on m.league_id = l.id
join Team T on T.team_api_id = M.home_team_api_id
;
""", conn)
qry19

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA,league_name,team_long_name
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20,Belgium Jupiler League,KRC Genk
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60,Belgium Jupiler League,SV Zulte-Waregem
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75,Belgium Jupiler League,KSV Cercle Brugge
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50,Belgium Jupiler League,KAA Gent
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67,Belgium Jupiler League,FCV Dender EH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25974,25975,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992091,10190,10191,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Switzerland Super League,FC St. Gallen
25975,25976,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992092,9824,10199,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Switzerland Super League,FC Vaduz
25976,25977,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992093,9956,10179,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Switzerland Super League,Grasshopper Club Zürich
25977,25978,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992094,7896,10243,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Switzerland Super League,Lugano


Q20. Complex Query to find the height distribution.

In [22]:
qry3 = pd.read_sql("""
select 
case when height>=150 and height<170 then '150/170'
when height>=170 and height<190 then '170/190'
when height>=190 then '190+'
end as height_distribution, count(player_api_id)
from player
group by 1
""", conn)

display(qry3)

,height_distribution,count(player_api_id)
0,150/170,192
1,170/190,9535
2,190+,1333
